In [1]:
import os, shutil
import tensorflow as tf
#Importing libraries to interact with Operating system


In [2]:
train_dir = os.path.join('../input/vggwallcrackuncrackdataset/train')
validation_dir = os.path.join('../input/vggwallcrackuncrackdataset/validation')
test_dir = os.path.join('../input/vggwallcrackuncrackdataset/test')

In [3]:
train_crack_dir = os.path.join('../input/vggwallcrackuncrackdataset/train/crack')
train_no_crack_dir = os.path.join('../input/vggwallcrackuncrackdataset/train/non crack')
validation_crack_dir = os.path.join('../input/vggwallcrackuncrackdataset/validation/crack')
validation_no_crack_dir = os.path.join('../input/vggwallcrackuncrackdataset/validation/non crack')
test_crack_dir = os.path.join('../input/vggwallcrackuncrackdataset/test/crack')
test_no_crack_dir = os.path.join('../input/vggwallcrackuncrackdataset/test/non crack')

In [4]:
print('total training crack  images:', len(os.listdir(train_crack_dir)))
print('total training no crack images:', len(os.listdir(train_no_crack_dir)))


print('total validation crack images:', len(os.listdir(validation_crack_dir)))
print('total validation no crack images:', len(os.listdir(validation_no_crack_dir)))

print('total test crack images:', len(os.listdir(test_crack_dir)))
print('total test no crack images:', len(os.listdir(test_no_crack_dir)))

total training crack  images: 1250
total training no crack images: 1250
total validation crack images: 500
total validation no crack images: 500
total test crack images: 500
total test no crack images: 500


In [5]:
from keras import layers
from keras import models
#Importing keras for preprocessing the data to feed to the network

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=20,
    class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(227, 227),
                                                        batch_size=1,
                                                        class_mode='categorical')
#Scaling the pixel values of images between 0 and 1
#taregt_size is The dimensions to which all images found will be resized
#batch_size denotes the subset size of training sample
#class_mode: classify between different categories like crack and no crack

Found 2500 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break
#Defining data per batch for training 

data batch shape: (20, 227, 227, 3)
labels batch shape: (20, 2)


In [8]:
IMG_SHAPE = (224, 224, 3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    
    
    
    classifier_activation="softmax",
    
)

In [9]:
feature_batch = base_model(data_batch)
print(feature_batch.shape)
#Dimension metrics 20x8x8

(20, 7, 7, 1280)


In [10]:
base_model.trainable = False

In [11]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(20, 1280)


In [13]:
prediction_layer = tf.keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(20, 2)


In [14]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [15]:
sgd = tf.keras.optimizers.SGD(lr=1e-6, decay=1e-6, momentum=1, nesterov=True)
model.compile(optimizer=sgd,loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [17]:
len(model.trainable_variables)

2

In [18]:
initial_epochs = 40
validation_steps=10

loss0,accuracy0 = model.evaluate(validation_generator, steps = validation_steps)

10/10 [==============================] - 0s 6ms/step - loss: 0.8243 - accuracy: 0.5000


In [19]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 0.82
initial accuracy: 0.50


In [20]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=20,
    class_mode='binary')

Found 2500 images belonging to 2 classes.


In [ ]:
X,Y = train_generator.next()

In [ ]:
Y

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset

# split into input (X) and output (Y) variables
X,Y = train_generator.next()
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    
    
    
    classifier_activation="softmax",
    
    )
    base_model.trainable = False
    feature_batch = base_model(data_batch)
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    feature_batch_average = global_average_layer(feature_batch)
    prediction_layer = tf.keras.layers.Dense(2)
    prediction_batch = prediction_layer(feature_batch_average)
    
  
    model = tf.keras.Sequential([base_model,global_average_layer,prediction_layer])

        
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    model.fit(X[train], Y[train], epochs=40, batch_size=2500, verbose=0)
    
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

In [ ]:
X

In [ ]:
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=validation_generator)
#Fitting the model in a CNN

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
#Plotting graphs b/w different parameters

In [ ]:


test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(200, 200),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy
test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)
predictions = model.predict_generator(test_generator, steps=1000)

predicted_classes = numpy.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)   

cm=confusion_matrix(true_classes,predicted_classes)
print(cm)

In [21]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=2500,
    class_mode='categorical')
X,Y = train_generator.next()


Found 2500 images belonging to 2 classes.


In [22]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

from sklearn.model_selection import KFold
import numpy as np

# Model configuration
batch_size = 50
img_width, img_height, img_num_channels = 227, 227, 3
loss_function = sparse_categorical_crossentropy
no_classes = 2
no_epochs = 25

verbosity = 1
num_folds = 10


(input_train, target_train)= train_generator.next()
(input_test, target_test) =  validation_generator.next()


input_shape = (img_width, img_height, img_num_channels)


input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data


# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = input_train
targets = target_train

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

In [23]:


for train, test in kfold.split(inputs, targets):
    
    model = model

  # Define the model architecture


  # Compile the model
    model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],batch_size=batch_size,epochs=no_epochs,verbose=verbosity)

  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    
    
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/25
45/45 [==============================] - 2s 45ms/step - loss: 4.4786 - accuracy: 0.4458
Epoch 2/25
45/45 [==============================] - 2s 48ms/step - loss: 2.5253 - accuracy: 0.6582
Epoch 3/25
45/45 [==============================] - 2s 46ms/step - loss: 0.8858 - accuracy: 0.9093
Epoch 4/25
45/45 [==============================] - 2s 45ms/step - loss: 0.4210 - accuracy: 0.9782
Epoch 5/25
45/45 [==============================] - 2s 47ms/step - loss: 0.4008 - accuracy: 0.9916
Epoch 6/25
45/45 [==============================] - 2s 46ms/step - loss: 0.2581 - accuracy: 0.9933
Epoch 7/25
45/45 [==============================] - 2s 48ms/step - loss: 0.1726 - accuracy: 0.9951
Epoch 8/25
45/45 [==============================] - 2s 46ms/step - loss: 0.1017 - accuracy: 0.9947
Epoch 9/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0571 - accuracy: 0.9960
Epoch 10/25


45/45 [==============================] - 2s 48ms/step - loss: 0.0689 - accuracy: 0.9956
Epoch 3/25
45/45 [==============================] - 2s 44ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 4/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 5/25
45/45 [==============================] - 2s 44ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 6/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 7/25
45/45 [==============================] - 2s 48ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 8/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 9/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0684 - accuracy: 0.9956
Epoch 10/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0685 - accuracy: 0.9956
Epoch 11/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0686 - accuracy: 0.9956
Epoch 12/25
45/45 [

45/45 [==============================] - 2s 45ms/step - loss: 0.0651 - accuracy: 0.9956
Epoch 5/25
45/45 [==============================] - 2s 44ms/step - loss: 0.0651 - accuracy: 0.9956
Epoch 6/25
45/45 [==============================] - 2s 47ms/step - loss: 0.0651 - accuracy: 0.9956
Epoch 7/25
45/45 [==============================] - 2s 47ms/step - loss: 0.0650 - accuracy: 0.9956
Epoch 8/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0650 - accuracy: 0.9956
Epoch 9/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0650 - accuracy: 0.9956
Epoch 10/25
45/45 [==============================] - 2s 44ms/step - loss: 0.0649 - accuracy: 0.9956
Epoch 11/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0649 - accuracy: 0.9956
Epoch 12/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0649 - accuracy: 0.9956
Epoch 13/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0649 - accuracy: 0.9956
Epoch 14/25
45/45

45/45 [==============================] - 2s 49ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 7/25
45/45 [==============================] - 2s 47ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 8/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 9/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 10/25
45/45 [==============================] - 2s 46ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 11/25
45/45 [==============================] - 2s 47ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 12/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 13/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 14/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 15/25
45/45 [==============================] - 2s 45ms/step - loss: 0.0648 - accuracy: 0.9956
Epoch 16/25
45/

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy= history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

In [ ]:

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(227, 227),
    batch_size=2500,
    class_mode='categorical')
X,Y = train_generator.next()



In [ ]:

# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# split into input (X) and output (Y) variables

# define 10-fold cross validation test harness
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    
  # create model
        model = model

        # Compile model
        model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
        # Fit the model
        model.fit(X[train], Y[train], epochs=25, batch_size=50, verbose=1)
        # evaluate the model
        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))